Celula 1:

In [34]:
import cv2 as cv
import numpy as np
import os

Celula 2:

In [35]:
# aplicare filtru pentru masa + decupare tabla de joc
def apply_filter_for_table(frame):

    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    low_bound = np.array([0, 0, 125])
    upp_bound = np.array([255, 255, 255])

    mask = cv.inRange(frame_hsv, low_bound, upp_bound)
    result = cv.bitwise_and(frame, frame, mask=mask)

    return mask, result

def extract_table(frame, mask):

    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    max_contour = max(contours, key=cv.contourArea)

    epsilon = 0.02 * cv.arcLength(max_contour, True)
    corners = cv.approxPolyDP(max_contour, epsilon, True)

    if len(corners) == 4:
        corners = corners.reshape(4, 2)
        sum_corners = corners.sum(axis=1)
        diff_corners = np.diff(corners, axis=1)

        top_left = corners[np.argmin(sum_corners)]
        bottom_right = corners[np.argmax(sum_corners)]
        top_right = corners[np.argmin(diff_corners)]
        bottom_left = corners[np.argmax(diff_corners)]

        width, height = 810, 810
        destination_corners = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype="float32")

        source_corners = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")
        M = cv.getPerspectiveTransform(source_corners, destination_corners)
        extracted_square = cv.warpPerspective(frame, M, (width, height))

        return extracted_square
    else:
        print("Error: Failed to detect 4 corners.")
        return None

Celula 3:

In [36]:
# aplicare filtru tabla + extragere careu de joc
def filter_and_extract_square(frame):

    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    lower_bound = np.array([25, 90, 0])
    upper_bound = np.array([255, 255, 255])

    mask = cv.inRange(frame_hsv, lower_bound, upper_bound)
    mask_inverted = cv.bitwise_not(mask)

    filtered_frame = cv.bitwise_and(frame, frame, mask=mask_inverted)

    gray = cv.cvtColor(filtered_frame, cv.COLOR_BGR2GRAY)
    _, binary = cv.threshold(gray, 50, 255, cv.THRESH_BINARY)
    binary = cv.dilate(binary, None, iterations=2) 
    binary = cv.erode(binary, None, iterations=1)

    contours, _ = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_contour = max(contours, key=cv.contourArea)
    epsilon = 0.05 * cv.arcLength(max_contour, True)
    corners = cv.approxPolyDP(max_contour, epsilon, True)

    if len(corners) == 4:
        corners = corners.reshape(4, 2)
        sum_corners = corners.sum(axis=1)
        diff_corners = np.diff(corners, axis=1)

        top_left = corners[np.argmin(sum_corners)]
        bottom_right = corners[np.argmax(sum_corners)]
        top_right = corners[np.argmin(diff_corners)]
        bottom_left = corners[np.argmax(diff_corners)]

        width, height = 810, 810
        destination_corners = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype="float32")
        source_corners = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")

        M = cv.getPerspectiveTransform(source_corners, destination_corners)
        extracted = cv.warpPerspective(frame, M, (width, height))

        return extracted
    else:
        print("Error: Failed to detect 4 corners.")
        return None


Celula 4:

In [37]:
# imparte imagiunea cu careul intr un vector de imagini cu fiecare celula
def split_into_cells(board_image):

    height, width = board_image.shape[:2]
    if height != 810 or width != 810:
        raise ValueError("Error: The image does not have the correct dimensions.")

    cells = []
    y = 0
    while y + 57.8571429 <= height + 1:
        row_cells = []
        x = 0
        while x + 57.8571429 <= width + 1:
            row_cells.append(board_image[int(round(y)):int(round(y + 57.8571429)), 
                                         int(round(x)):int(round(x + 57.8571429))])
            x += 57.8571429
        cells.append(row_cells)
        y += 57.8571429
    
    return cells

# verificam daca pozitia este favorabila = exista cel putin un nr != -1 vecin cu acea pozitie
def is_favorable_position(row, col, position_matrix):
    rows = len(position_matrix)
    cols = len(position_matrix[0])

    if row < 0 or row >= rows or col < 0 or col >= cols:
        return False

    if position_matrix[row][col] != -1:
        return False 

    neighbors = [
        (row - 1, col),
        (row + 1, col),
        (row, col - 1),
        (row, col + 1),
    ]

    for nr, nc in neighbors:
        if 0 <= nr < rows and 0 <= nc < cols:
            if position_matrix[nr][nc] != -1:
                return True

    return False

Celula 5:

In [38]:

# taiem putin din margini inate sa aplicam filtrul pe piese
def trim_edges(image, top=7, bottom=7, left=7, right=5):
    h, w = image.shape[:2]
    return image[top:h-bottom, left:w-right]

# filtru care imi face fundalul piesei neagru si cifra alba
def apply_filter_for_token(image):
    if len(image.shape) == 2:
        image = cv.cvtColor(image, cv.COLOR_GRAY2BGR)
        
    hsv_image = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    lower_hsv = np.array([0, 0, 80])
    upper_hsv = np.array([180, 255, 255])

    mask = cv.inRange(hsv_image, lower_hsv, upper_hsv)
    binary_mask = cv.bitwise_not(mask)

    return binary_mask

# decuparea posei cu piesa astfel incat sa ramana doar numarul si cat mai putin din fundal
def crop_token(original_image, binary_mask):

    contours, _ = cv.findContours(binary_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        print("Error: No contour was detected.")
        return original_image

    image_with_contours = original_image.copy()
    cv.drawContours(image_with_contours, contours, -1, (0, 255, 0), 2)

    x_min = y_min = float('inf')
    x_max = y_max = 0

    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        
        x_min = min(x_min, x)
        y_min = min(y_min, y)
        x_max = max(x_max, x + w)
        y_max = max(y_max, y + h)

    cropped_image = original_image[y_min:y_max, x_min:x_max]

    return cropped_image

Celula 7:

In [39]:
# cauta vecini valizi pentru calcularea numerelor pozibile care poti fi la pozitia mea
def find_valid_neighbors(row, col, position_matrix):
    neighbors = [
        (-1, 0), (1, 0),
        (0, -1), (0, 1),
    ]
    valid_neighbors = []
    
    for dr, dc in neighbors:
        nr, nc = row + dr, col + dc
        if 0 <= nr < len(position_matrix) and 0 <= nc < len(position_matrix[0]) and position_matrix[nr][nc] >= 0:
            line = [(position_matrix[nr][nc])]
            nnr, nnc = nr + dr, nc + dc
            if 0 <= nnr < len(position_matrix) and 0 <= nnc < len(position_matrix[0]) and position_matrix[nnr][nnc] >= 0:
                line.append((position_matrix[nnr][nnc]))
    
            valid_neighbors.append(line)
    
    return valid_neighbors

# calculam toate numerele posibile care pot fi la pozitia mea 
def calculate_operations(p1, p2):
    results = set()

    results.add(p1 + p2)
    results.add(abs(p1 - p2))
    results.add(abs(p2 - p1))
    results.add(p1 * p2)
    if p2 != 0 and p1 % p2 == 0:
        results.add(p1 // p2)
    if p1 != 0 and p2 % p1 == 0:
        results.add(p2 // p1)

    results = list(results)
    return results

# comparam piesa pusa acum pe tabla cu piesele posibile din fisierul meu de cifre
def find_best_match(calculated_values, template_folder, cell_image):
    cell_image = trim_edges(cell_image, top=7, bottom=7, left=7, right=5)
    binary_mask = apply_filter_for_token(cell_image)
    cell_image = crop_token(cell_image, binary_mask)

    min_difference = float('inf')
    best_value = None
    for value in calculated_values:
        template_path = os.path.join(template_folder, f"{value}.png")
        if not os.path.exists(template_path):
            continue
        template = cv.imread(template_path, cv.IMREAD_GRAYSCALE)
        resized_template = cv.resize(template, (cell_image.shape[1], cell_image.shape[0]))
        diff = cv.absdiff(cell_image, resized_template)
        score = np.sum(diff)
        if score < min_difference:
            min_difference = score
            best_value = value
    return best_value, min_difference


Celula 8:

In [40]:
initial_position_matrix = [
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, 1, 2, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, 3, 4, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
    [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
]

position_matrix = [row[:] for row in initial_position_matrix]

# resetarea matricei pentru cand se incepe un nou joc
def reset_position_matrix():
    global position_matrix
    position_matrix = [row[:] for row in initial_position_matrix]

# realizarea vectorului de imagini pe care le vom compara doua cate doua
def create_image_vector(training_folder, auxiliary_image_path):
    training_images = sorted([file for file in os.listdir(training_folder) if file.endswith(".jpg")])
    auxiliary_image = cv.imread(auxiliary_image_path)
    max_length = len(training_images) + 4

    image_vector = []

    for i in range(max_length):
        if i in {0, 51, 102, 153}:
            image_vector.append(auxiliary_image)
        else:
            idx = i - sum(1 for pos in [0, 51, 102, 153] if pos < i)
            if idx < len(training_images):
                image_path = os.path.join(training_folder, training_images[idx])
                image_vector.append(cv.imread(image_path))
            else:
                break

    return image_vector, training_images


Celula 9:

In [41]:
def compare_cells(cells_img1, cells_img2, position_matrix, template_folder):
    rows = len(cells_img1)
    cols = len(cells_img1[0])
    
    differences = []

    for row in range(rows):
        for col in range(cols):
            gray1 = cv.cvtColor(cells_img1[row][col], cv.COLOR_BGR2GRAY)
            gray2 = cv.cvtColor(cells_img2[row][col], cv.COLOR_BGR2GRAY)
        
            diff = cv.absdiff(gray1, gray2)
            _, diff_binary = cv.threshold(diff, 50, 255, cv.THRESH_BINARY)
            difference_value = cv.countNonZero(diff_binary)
            differences.append((difference_value, (row, col), gray2))

    differences.sort(reverse=True, key=lambda x: x[0])

    for difference_value, (row, col), cell in differences:
        if is_favorable_position(row, col, position_matrix):

            neighbors = find_valid_neighbors(row, col, position_matrix)

            best_value = None
            min_difference = 99999999

            for line in neighbors:
                if len(line) >= 2:
                    p1 = line[0]
                    p2 = line[1]

                    calculated_values = calculate_operations(p1, p2)
                    current_best_value, match_difference = find_best_match(calculated_values, template_folder, cell)

                    if current_best_value is not None and match_difference < min_difference:
                        best_value = current_best_value
                        min_difference = match_difference

            if best_value is not None:
                position_matrix[row][col] = best_value
                return (row, col), difference_value, best_value


    return None, 0, 0

Celula 10: Task1 + Task2

In [42]:
training_folder = "antrenare" 
auxiliary_image_path = "imagini_auxiliare/01.jpg"
template_folder = "cifre"

output_dir = "fisiere_solutie/464_Saicu_Carina"
os.makedirs(output_dir, exist_ok=True)

auxiliary_img = cv.imread(auxiliary_image_path)

image_vector, training_image_names = create_image_vector(training_folder, auxiliary_image_path)

files = sorted(os.listdir(training_folder))

grouped_files = {}
for file in files:
    if file.endswith(".jpg") or file.endswith(".txt"):
        prefix = file.split("_")[0]
        if prefix not in grouped_files:
            grouped_files[prefix] = []
        grouped_files[prefix].append(file)

for prefix, file_group in grouped_files.items():
    images = sorted([f for f in file_group if f.endswith(".jpg")])
        
    file_index = 1

    first_image_path = os.path.join(training_folder, images[0])
    first_image = cv.imread(first_image_path)

    mask_aux, filtered_aux = apply_filter_for_table(auxiliary_img)
    square_aux = extract_table(auxiliary_img, mask_aux)
    large_square_aux = filter_and_extract_square(square_aux)

    mask_first, filtered_first = apply_filter_for_table(first_image)
    square_first = extract_table(first_image, mask_first)
    large_square_first = filter_and_extract_square(square_first)

    if large_square_aux is None or large_square_first is None:
        result = f"Error: The game board could not be extracted from the image without pieces or from {images[0]}.\n"
    else:

        cells_aux = split_into_cells(large_square_aux)
        cells_first = split_into_cells(large_square_first)


        max_diff_cell, max_difference, best_value = compare_cells(cells_aux, cells_first, position_matrix, template_folder)

        if max_diff_cell is not None:
            row_label = max_diff_cell[0] + 1
            col_label = chr(65 + max_diff_cell[1])

            result = f"{row_label}{col_label} {best_value}"
        else:
            result = f"No significant differences were found between the image without pieces and {images[0]}.\n"

    output_file_name = f"{prefix}_{str(file_index).zfill(2)}.txt"
    with open(os.path.join(output_dir, output_file_name), "w", encoding="utf-8") as output_file:
        output_file.write(result)
    file_index += 1

    for i in range(len(images) - 1):
        img1_path = os.path.join(training_folder, images[i])
        img2_path = os.path.join(training_folder, images[i + 1])

        img1 = cv.imread(img1_path)
        img2 = cv.imread(img2_path)

        mask1, filtered1 = apply_filter_for_table(img1)
        square1 = extract_table(img1, mask1)
        large_square1 = filter_and_extract_square(square1)

        mask2, filtered2 = apply_filter_for_table(img2)
        square2 = extract_table(img2, mask2)
        large_square2 = filter_and_extract_square(square2)

        if square1 is None or square2 is None:
            result = f"Error: The game board could not be extracted from {images[i]} or {images[i + 1]}.\n"
            continue

        cells1 = split_into_cells(large_square1)
        cells2 = split_into_cells(large_square2)

        max_diff_cell, max_difference, best_value = compare_cells(cells1, cells2, position_matrix, template_folder)

        if max_diff_cell is not None:
            row_label = max_diff_cell[0] + 1
            col_label = chr(65 + max_diff_cell[1])

            result = f"{row_label}{col_label} {best_value}"

            output_file_name = f"{prefix}_{str(file_index).zfill(2)}.txt"
            with open(os.path.join(output_dir, output_file_name), "w", encoding="utf-8") as output_file:
                output_file.write(result)

            file_index += 1

        if i == 48:
            reset_position_matrix()

Celula 11: Task3

In [43]:
def calculate_scores():
    solution_folder = "fisiere_solutie/464_Saicu_Carina"
    training_folder = "antrenare"
    output_folder = "fisiere_solutie/464_Saicu_Carina"
    os.makedirs(output_folder, exist_ok=True)

    multiply_by_3 = {"1A", "1G", "1H", "1N", "7A", "7N", "8A", "8N", 
                     "14A", "14G", "14H", "14N"}
    multiply_by_2 = {"2B", "2M", "3C", "3L", "4D", "4K", "5E", "5J", 
                     "10E", "10J", "11D", "11K", "12C", "12L", "13B", "13M"}

    for game_number in range(1, 5):
        turns_file = os.path.join(training_folder, f"{game_number}_turns.txt")
        scores_file = os.path.join(output_folder, f"{game_number}_scores.txt")

        if not os.path.exists(turns_file):
            print(f"The file {turns_file} does not exist, skipping this game.")
            continue

        with open(turns_file, "r") as file:
            turns = file.readlines()

        scores = []
        cumulative_score = 0

        for i, turn in enumerate(turns):
            player, start_file = turn.strip().split()
            start_file = int(start_file)

            if i + 1 < len(turns):
                _, next_start_file = turns[i + 1].strip().split()
                end_file = int(next_start_file) - 1
            else:
                end_file = start_file
                while os.path.exists(os.path.join(solution_folder, f"{game_number}_{str(end_file + 1).zfill(2)}.txt")):
                    end_file += 1

            for file_number in range(start_file, end_file + 1):
                solution_file = os.path.join(solution_folder, f"{game_number}_{str(file_number).zfill(2)}.txt")
                if not os.path.exists(solution_file):
                    print(f"The file {solution_file} does not exist, moving past.")
                    continue

                with open(solution_file, "r") as sol_file:
                    content = sol_file.read().strip()
                    try:
                        position, value = content.split()
                        value = int(value)

                        if position in multiply_by_3:
                            value *= 3
                        elif position in multiply_by_2:
                            value *= 2

                        cumulative_score += value
                    except ValueError:
                        print(f"Error parsing the file {solution_file}, skipping it.")
                        continue

            scores.append(f"{player} {start_file} {cumulative_score}")
            cumulative_score = 0

        with open(scores_file, "w", encoding="utf-8") as output_file:
            output_file.write("\n".join(scores))

        print(f"The scores for game {game_number} have been saved to {scores_file}.")

if __name__ == "__main__":
    calculate_scores()


The scores for game 1 have been saved to fisiere_solutie/464_Saicu_Carina\1_scores.txt.
The scores for game 2 have been saved to fisiere_solutie/464_Saicu_Carina\2_scores.txt.
The scores for game 3 have been saved to fisiere_solutie/464_Saicu_Carina\3_scores.txt.
The scores for game 4 have been saved to fisiere_solutie/464_Saicu_Carina\4_scores.txt.


Celula 12: codul de evaluare

In [44]:
def compare_annotations_position_token(filename_predicted,filename_gt,verbose=0):
	p = open(filename_predicted,"rt")  	
	gt = open(filename_gt,"rt") 

	
	line_gt = gt.readline()
	current_pos_gt, current_token_gt = line_gt.split()
	if verbose:
		print(current_pos_gt,current_token_gt)
	
	line_p = p.readline()
	current_pos_p, current_token_p = line_p.split()
	if verbose:
		print(current_pos_p,current_token_p)

	
	match_positions = 1
	match_tokens = 1
	if(current_pos_p != current_pos_gt):
		match_positions = 0
	if(current_token_p != current_token_gt):
		match_tokens = 0
	

	points_positions = 0.025 * match_positions
	points_tokens = 0.01 * match_tokens	


	return points_positions, points_tokens

#change this on your machine pointing to your results (txt files)
predictions_path_root = "fisiere_solutie/464_Saicu_Carina/"

#change this on your machine to point to the ground-truth test
gt_path_root = "aaa/" 



def compare_annotations_score(filename_predicted,filename_gt,verbose=0):
	p = open(filename_predicted,"rt")  	
	gt = open(filename_gt,"rt") 

	all_lines_gt = gt.readlines()
	all_lines_p = p.readlines()


	number_lines_gt = len(all_lines_gt)

	points_scores = 0

	try:
		for i in range(0,number_lines_gt):
			line_gt = all_lines_gt[i]
			line_p = all_lines_p[i]
			if (line_gt==line_p):
				points_scores += 0.04
	except:
		pass

	return points_scores


#change this to 1 if you want to print results at each turn
verbose = 1
total_points = 0
for game in range(1,5):
# change this for game in range(1,5):
	points_score = 0
	for turn in range(1,51):
		
		name_turn = str(turn)
		if(turn < 10):
			name_turn = '0'+str(turn)

		filename_predicted = predictions_path_root + str(game) + '_' + name_turn + '.txt'
		filename_gt = gt_path_root + str(game) + '_' + name_turn + '.txt'

		game_turn = str(game) + '_' + name_turn
		points_position = 0
		points_tokens = 0

		try:
			points_position, points_tokens  = compare_annotations_position_token(filename_predicted,filename_gt,verbose)
		except:
			print("Pentru imaginea: ", game_turn, " functia de evaluare a intampinat o eroare")

		print("Imaginea: ", game_turn, "Puncte task 1: ", points_position, "Puncte task 2: ",points_tokens)
		
		total_points = total_points + points_position + points_tokens

	filename_predicted_scores = predictions_path_root + str(game) + '_scores' + '.txt'
	filename_gt_scores = gt_path_root + str(game) + '_scores' + '.txt'
	points_score = compare_annotations_score(filename_predicted_scores,filename_gt_scores,verbose=0)
	print("Puncte task 3 ", points_score)

	total_points = total_points + points_score
	print("Total puncte = ", total_points)


print(total_points)

9H 8
9H 8
Imaginea:  1_01 Puncte task 1:  0.025 Puncte task 2:  0.01
10H 32
10H 32
Imaginea:  1_02 Puncte task 1:  0.025 Puncte task 2:  0.01
11H 40
11H 40
Imaginea:  1_03 Puncte task 1:  0.025 Puncte task 2:  0.01
12H 72
12H 72
Imaginea:  1_04 Puncte task 1:  0.025 Puncte task 2:  0.01
6H 8
6H 8
Imaginea:  1_05 Puncte task 1:  0.025 Puncte task 2:  0.01
5H 10
5H 10
Imaginea:  1_06 Puncte task 1:  0.025 Puncte task 2:  0.01
4H 80
4H 80
Imaginea:  1_07 Puncte task 1:  0.025 Puncte task 2:  0.01
3H 8
3H 8
Imaginea:  1_08 Puncte task 1:  0.025 Puncte task 2:  0.01
2H 10
2H 10
Imaginea:  1_09 Puncte task 1:  0.025 Puncte task 2:  0.01
1H 18
1H 18
Imaginea:  1_10 Puncte task 1:  0.025 Puncte task 2:  0.01
6G 3
6G 3
Imaginea:  1_11 Puncte task 1:  0.025 Puncte task 2:  0.01
6F 11
6F 11
Imaginea:  1_12 Puncte task 1:  0.025 Puncte task 2:  0.01
6E 14
6E 14
Imaginea:  1_13 Puncte task 1:  0.025 Puncte task 2:  0.01
6I 5
6I 5
Imaginea:  1_14 Puncte task 1:  0.025 Puncte task 2:  0.01
6J 3
6J 3
